In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [26]:
df = pd.read_csv('/Users/hrishityelchuri/Desktop/github/grabhack/driver-credit-scoring-engine/data/raw/partners.csv')

In [27]:
df

,partner_id,role,gender,age_group,city_tier,tenure_months,days_active,trips_per_week,earnings_avg,earnings_var,...,cancel_rate,customer_rating,complaints,accidents,night_shift_pct,cashless_ratio,wallet_txn_volume,vehicle_age,past_due_history,good_repayment
0,1,driver,M,<=25,2,36,19,52,638.443420,101.826834,...,0.135795,4.837981,0,0,0.348988,0.350500,160.028448,4.336435,0,1
1,2,merchant,M,<=25,2,17,24,53,444.440684,69.151447,...,0.061321,4.778687,0,0,0.286113,0.915312,104.718719,5.482631,0,1
2,3,merchant,M,26-35,1,61,28,35,659.666189,223.266722,...,0.045608,4.418110,1,0,0.287171,0.642508,134.089531,6.072208,0,1
3,4,driver,F,<=25,3,33,18,58,503.720971,24.461841,...,0.053521,4.782128,0,0,0.279957,0.671180,268.472293,3.909398,0,0
4,5,driver,M,50+,3,27,19,57,637.742060,150.778596,...,0.150525,4.905054,0,0,0.108403,0.785409,0.000000,3.512688,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7996,driver,M,50+,2,22,26,37,651.340236,178.112415,...,0.009458,4.877334,0,0,0.687393,0.451506,275.217454,4.551144,0,1
7996,7997,driver,M,26-35,3,12,23,31,804.831806,21.553709,...,0.109542,4.864437,0,0,0.562117,0.441517,323.231347,0.000000,1,1
7997,7998,driver,M,26-35,2,13,21,44,648.224239,48.960279,...,0.130103,4.602213,0,0,0.193823,0.741244,240.555107,1.850392,0,1
7998,7999,merchant,M,26-35,1,45,20,81,861.418504,49.013844,...,0.124617,4.975657,0,0,0.212527,0.836395,352.932530,4.547698,0,1


In [28]:
def cap_outliers(series, lower_quantile=0.01, upper_quantile=0.99):
    """Clip values outside [q1, q99] to reduce outlier impact."""
    lower = series.quantile(lower_quantile)
    upper = series.quantile(upper_quantile)
    return series.clip(lower, upper)

In [29]:
def log_transform(series):
    """Log-transform skewed features (add 1 to handle zeros)."""
    return np.log1p(series)

In [30]:
def clean_partner_data(df):
    df = df.copy()

    # Outlier capping
    df["trips_per_week"] = df["trips_per_week"].clip(0, 80)
    df["complaints"] = cap_outliers(df["complaints"])
    df["earnings_var"] = log_transform(cap_outliers(df["earnings_var"], 0.01, 0.95))

    return df

In [31]:
def reweight_samples(df, sensitive_col="city_tier"):
    """
    Compute sample weights inversely proportional to group frequency.
    This helps in fairness-aware model training.
    """
    group_counts = df[sensitive_col].value_counts()
    weights = df[sensitive_col].map(lambda x: 1.0 / group_counts[x])
    weights = weights / weights.mean()  # normalize
    return weights

In [32]:
df_clean = clean_partner_data(df)
sample_weights = reweight_samples(df_clean, sensitive_col="city_tier")

In [33]:
df_clean.to_csv("/Users/hrishityelchuri/Desktop/github/grabhack/driver-credit-scoring-engine/data/processed/df_clean.csv", index=False)